In [3]:
#DLSProcessing
#Adapting old DLS python scripts for UTSA data and RapfLab generally
#Rebecca Rapf
#June 24, 2021


import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import glob
import os
import pandas as pd
#import datetime
#Note -- if you change anything in the DLSFunctionLibrary code, you need to close and 
#re-initialize this notebook for it to update
import DLSFunctionLibrary as DFL


Path0 = '/Users/rebeccarafp/Documents/RapfLabPython/DLS/210617/CorrFunctions/'
#SAVE_DATA = 'yes'
SAVE_DATA = 'yes'
FileType = "csv"
DLSType = "UTSA"

##### CONSTANTS -- specific for each DLS Instrument/experimental set up #####
TEMP = 298 #K
LASER = 640 #nm
VISC = 1 #cP
####################
AvgRadii = pd.DataFrame(index = ["Avg","Std","AvgR2"])

os.chdir(Path0)
print(os.getcwd())

#Make the output folder if it does not already exist
outputfolder = "Fitted"
isdir = os.path.isdir(outputfolder)
if isdir == False:
    os.mkdir(outputfolder)
    
#Read in all files of a given file type
ListFiles = sorted(glob.glob("*"+FileType))
ListFiles.sort(key=os.path.getmtime)
print(ListFiles)


#Read in all files
for entry in range(len(ListFiles)):
    
    p_file = ListFiles[entry]
    print(p_file)
    
    #strip out name of experiment
    prename = p_file[:-(len(FileType)+1)]
    #print(prename)

    if DLSType == "UTSA":
        [data,num_meas] = DFL.ReadUTSA(p_file)
    else:
        print("You need to use a different program that can read data for a diff DLS")

    
    #Initializing the parameters for fitting function -- takes temperature in Kelvin 
    #and laser wavelength in nm as arguments
    modelParamsDict = DFL.initializeExponentialModelParams(TEMP,LASER,VISC);
    #print(modelParamsDict)
    
    
    #Initializing for Normalizing Correlation Functions ###
    
    ###Check if correlation functions are already normalized ####
    if  0.99 <= data.iloc[:,1][1:25].mean() <= 1.1:
        print("Already Normalized")
        #NormData = data
        print(data.iloc[:,1][1:25].mean())
    else:
        NormData = pd.DataFrame()    
        SumNormCorr = np.zeros(len(data))
        SumUnNormCorr = np.zeros(len(data))
        SumNormAvg = 0
    
        for entry in range(num_meas):
            if entry == 0:
                if DLSType == "UTSA":
                    NormData[prename+"DecayTime"] = data.iloc[:,0]/1000 #convert to ms
            #print(CleanedData)

            CopyArray =  data.iloc[:,entry+1]
            NormAvg = CopyArray[1:25].mean()
        #print(NormAvg)
            NormCopyArray = CopyArray.div(NormAvg)
            NormData[prename+"Corr"+str(entry)] = NormCopyArray
        #print(NormData.iloc[:,1][1:25].mean())

            SumNormAvg += NormAvg

            SumUnNormCorr += CopyArray
            SumNormCorr += NormCopyArray  

        AvgNormCorr = SumNormCorr/num_meas

        AvgUnNormCorr = SumUnNormCorr/SumNormAvg

        NormData[prename+"AvgNormCorr"] = AvgNormCorr
        NormData[prename+"AvgUnNormCorr"] = AvgUnNormCorr
    
        NormData.to_csv("Norm/"+prename+'CleanCorr.csv')
    
    #Initialize data frames and array for fitting process
    FitArray = pd.DataFrame()
    R2Array = np.zeros(num_meas)
    RadiiArray = np.zeros(num_meas)
    RadVarArray = np.zeros(num_meas)
    RadStdArray = np.zeros(num_meas)
    
    #IndexNames = range(num_meas-2)#+["AvgNormCorr:","AvgUnNormCorr"]
    #print(IndexNames)
    #print(NormData.head())
    
    for entry in range(num_meas):
        #print(entry)
        Decays = NormData.iloc[:,0]#/1000 #units needs to be in ms. Plate Reader Data is in us.
        #print(Decays)
        
        Corr = NormData.iloc[:,entry+1]
        #print(Corr)
        CurrName = NormData.columns[entry+1]
    
        modelParamsDict["DecayTime"] = Decays
        #print AvgData.iloc[:,i]
        Radius, RadVar = curve_fit(DFL.ExponentialModel, modelParamsDict, Corr, maxfev = 100000)
        #print(Radius)
        RadiiArray[entry] = Radius
        RadVarArray[entry] = RadVar
        RadStdArray[entry] = np.sqrt(RadVar)
        #print(RadiiArray)
    
        FitArray['Fit'+CurrName] = (DFL.ExponentialModel(modelParamsDict,Radius))
    #    
        R0 = np.sum((DFL.ExponentialModel(modelParamsDict,Radius) - Corr)**2)
    #    #print AvgR0
        R1 = np.sum((Corr - np.mean(Corr))**2)
        R2 = 1 - (R0/R1)
        R2Array[entry] = R2
        
        #print(R2)
    
    ExtractedRadii = pd.DataFrame()#index = IndexNames)
    
    ExtractedRadii[prename+'Radii']=RadiiArray
    ExtractedRadii[prename+'RadVar']=RadVarArray
    ExtractedRadii[prename+'RadStd']=RadStdArray
    ExtractedRadii[prename+'R2']=R2Array
    #
    #print(ExtractedRadii)
    
    AvgRadii[prename+"AvgRadii"] = [RadiiArray.mean(),RadiiArray.std(),R2Array.mean()]
    #print(AvgRadii)
    #AvgRadii[prename+"AvgRadiiStd"] = []
    #print(AvgRadiiStd)
    
    if SAVE_DATA == 'yes':
        FitArray.to_csv("./"+outputfolder+"/"+prename+'Fits.csv')
        ExtractedRadii.to_csv("./"+outputfolder+"/"+prename+'Radii.csv')
    else:
        print("DATA WAS NOT SAVED")
if SAVE_DATA == "yes":
    AvgRadii.to_csv("./"+outputfolder+"/"+"SummaryRadii.csv")

/Users/rebeccarafp/Documents/RapfLabPython/DLS/210617/CorrFunctions
['210616OOApost.csv', '210612OICApost.csv', '210610OVApost.csv', '210615OBApost.csv', '210610OVApost_2.csv', '210616OOApre.csv', '210612OICApre.csv', '210610OVApre.csv', '210615OBApre.csv']
210616OOApost.csv
210612OICApost.csv
210610OVApost.csv
210615OBApost.csv
210610OVApost_2.csv
210616OOApre.csv
210612OICApre.csv
210610OVApre.csv
210615OBApre.csv
